# Item-based collaborative filtering and parameter selection for finding nearest neighbors

In this exercise, we will learn **user-based collaborative filtering**, which focuses on the similarity between items.
Also, we will learn and discuss parameters to select nearest neighbors (similar items).

In the exercise, we will use the following python libraries.
Let's install them before starting the exercise.


* numpy、scipy
    * Basic libraries for data science
* pandas
    * A library to efficient handle table-like data
* plotly
    * A library to make interactive charts

To the above libraries, run the following commands on Jupyter.

```
!pip install numpy
!pip install scipy
!pip install pandas
!pip install plotly
```

After the installation, if the following commands provide no errors, then you have succeeded in the installation.

```
import numpy
import pandas
import scipy
import plotly
```

---
## Load libraries and data
In this exercise, we will use the simple data being used in the last exercise and apply item-based filtering to the data.
Before starting the exercise, let's load necessary libraries by running the following commands.

In [1]:
import numpy as np
import pandas as pd

Let's load the data. The data we will use for this exercise is located [here](https://raw.githubusercontent.com/trycycle/recommender-system-2020/main/data/small-example.tsv).
The filename is **small-example.csv**. The data is the same as the one that we used last week.
In this file, each row means each user's ratings to all items. Each rating score to each item is separated by commas.
Please note that the first line is a header.

Run the following code, and then we can download the file and load it into the variable `df`.

In [2]:
# The parameter index_col=0 enables you to set the first column on data as index names
url = "https://raw.githubusercontent.com/trycycle/recommender-system-2020/main/data/small-example.tsv"
df = pd.read_table(url, sep="\t" ,index_col=0)
df

,item1,item2,item3,item4,item5
user,,,,,
Alice,5,3,4,4,NaN
user1,3,1,2,3,3.0
user2,4,3,4,3,5.0
user3,3,3,1,5,4.0
user4,1,5,5,2,1.0


Using `read_csv` method of `pandas`, we can load data as a **data frame** object.
We can see that each row means each user and each column means each item rating on the variable `df` (the data frame object).

## Calculation similarity between items

In the last exercise, we calculated similarities between users to do user-based collaborative filtering.
Here, let's calculate similarities **between items** and do item-based collaborative filtering.

There are several choices for item similarity metrics, such as Euclidean distance and Pearson correlation coefficient.
However, as we learned in the lecture, **cosine similarity** is known as the best metric to compute the similarity between items for item-based collaborative filtering.

Let's calculate cosine similarity between items.
Given two vectors $v_1$ and $v_2$, the cosine similarity $sim(v_1, v2)$ of $v_1$ and $v_2$ is defined as follows:

\begin{equation*}
sim(v_1, v_2) = \frac{v_1 \cdot v_2}{|v_1| |v_2|}
\end{equation*}

Here, $a \cdot b$ is the inner product of vectors $a$ and $b$, and $|a|$ is the L2 norm (length) of vector $a$.

The following `calc_cosine_similarity` is a function to calculate cosine similarity.
Let's use it for this practice.
The function `calc_cosine_similarity`'s input is a numpy matrix object.
If a matrix is an input to the function, the function considers that each row data of the matrix as vector data and calculate cosine similarities between all row vectors.


In [3]:
def calc_cosine_similarity(M):
    """ This function ignores N/A elements in M and calculate cosine similarities between all rows.
    """
    if type(M) == np.ndarray:
        row_num, col_num = M.shape
        
        if row_num >= 2:
            sim_matrix = np.empty((row_num, row_num))
            sim_matrix[:, :] = np.nan

            for i in range(row_num):
                for j in range(i, row_num):
                    if i == j:
                        sim_matrix[i, j] = 1
                    else:
                        inner_product = np.nansum(M[i, :] * M[j, :])
                        
                        target_cols = ~np.isnan(M[i, :]) & ~np.isnan(M[j, :])
                        
                        norm_i = np.linalg.norm(M[i, :][target_cols])
                        norm_j = np.linalg.norm(M[j, :][target_cols])
                        
                        sim_matrix[i, j] = inner_product / (norm_i * norm_j)
                        sim_matrix[j, i] = sim_matrix[i, j]

            return sim_matrix
    return None  

For cosine similarity calculation, we convert the dataframe `df`, which contains rating scores of users, into a numpy matrix object.

In [4]:
rating_matrix = df.values
rating_matrix

array([[ 5.,  3.,  4.,  4., nan],
       [ 3.,  1.,  2.,  3.,  3.],
       [ 4.,  3.,  4.,  3.,  5.],
       [ 3.,  3.,  1.,  5.,  4.],
       [ 1.,  5.,  5.,  2.,  1.]])

As we can see the output data, each row of `rating matrix` is correspondent to each user and each column is correspondent to each item.

Let's calculate cosine similarities between items, by using this `rating_matrix` and the function `calc_cosine_similarity`.
As we already learned, the function `calc_cosine_similarity` computes similarities between row vectors.
Each row of `rating_matrix` is correspondent to each user, and so we will input the transposed matrix of `rating_matrix`.

In [5]:
calc_cosine_similarity(rating_matrix.T)

array([[1.        , 0.78025959, 0.81978229, 0.94337007, 0.99410024],
       [0.78025959, 1.        , 0.94201969, 0.84798442, 0.73885058],
       [0.81978229, 0.94201969, 1.        , 0.78402509, 0.72261012],
       [0.94337007, 0.84798442, 0.78402509, 1.        , 0.93955848],
       [0.99410024, 0.73885058, 0.72261012, 0.93955848, 1.        ]])

We have obtained cosine similarities between items!

However, this calculation does not consider the difference in the average rating behavior of the users.
That is, the system cannot capture such tendency as some users grade items easily and other users grade ones strictly.

To take this problem into account, let's adjust rating scores of users.
For this adjustment, here we subtract the user average from the ratings.
We calculate the average score of each user as follows.

In [6]:
df.mean(axis=1) # Calculate the averages by row. In this calculation, N/A data is ignored.

user
Alice    4.0
user1    2.4
user2    3.8
user3    3.2
user4    2.8
dtype: float64

For easier matrix operation, we convert the calculated averages into a numpy matrix object.

In [7]:
mean_vec = df.mean(axis=1).values # Convert data to a numpy vector data
mean_vec = np.reshape(mean_vec, (5, 1)) # Convert a vector to a 5x1 matrix data
mean_vec

array([[4. ],
       [2.4],
       [3.8],
       [3.2],
       [2.8]])

Now we have average score vectors.
Let's adjust rating scores as follows.

In [8]:
mod_rating_matrix = rating_matrix - mean_vec
mod_rating_matrix.T # Transpose a matrix so that each row means each item and each row means each user

array([[ 1. ,  0.6,  0.2, -0.2, -1.8],
       [-1. , -1.4, -0.8, -0.2,  2.2],
       [ 0. , -0.4,  0.2, -2.2,  2.2],
       [ 0. ,  0.6, -0.8,  1.8, -0.8],
       [ nan,  0.6,  1.2,  0.8, -1.8]])

We can calculate adjusted cosine similarities between items (**item similarity matrix**), by applying `calc_cosine_similarity` to the adjusted rating matrix `mod_rating_matrix`.


In [9]:
sim_matrix = calc_cosine_similarity(mod_rating_matrix.T)
sim_matrix

array([[ 1.        , -0.93972516, -0.54706829,  0.26784105,  0.80491448],
       [-0.93972516,  1.        ,  0.62054308, -0.36064519, -0.90823188],
       [-0.54706829,  0.62054308,  1.        , -0.88137969, -0.76356039],
       [ 0.26784105, -0.36064519, -0.88137969,  1.        ,  0.43306269],
       [ 0.80491448, -0.90823188, -0.76356039,  0.43306269,  1.        ]])

## Predicts item ratings using item similarity
Let's use the above item similarity matrix and predict Alice ($u_a$)'s rating score for item 5 ($i_5$).
Our approach is as follows:
1. We define that the top 2 similar items in Alice's rated items are nearest neighbor items ($i \in I_s$).
2. Use the following equation to calculate the rating score of Alice for item 5

\begin{equation}
rating(u_a, i_5) = \overline{r_{u_a}} + \frac{\sum_{i \in I_s}sim(i_5, i) \times r_{u_a, i}}{\sum_{i \in I_s}sim(i_5, i)}
\end{equation}

Here, $\overline{r_u}$ means $u$'s average rating score, and $r'(u, i)$ is the adjusted rating score of user $u$ for item $i$.
$sim(i_x, i_y)$ is the cosine similarity between items $x$ and $y$.

Let's predict Alice's rating score for item 5.
At first, we will obtain the similarity of the two nearest neighbor items for item 5.
As we can see the item similarity matrix, we can find that the most two similar items for item 5 are item 1 (sim=0.805) and item 4 (sim=0.433).
We can obtain the similarity scores of these two items by using the below code.


In [10]:
# In sim_matrix, item 5 is correspondent to the 4th row (column).
sim_vec = sim_matrix[4, [0,3]]
sim_vec

array([0.80491448, 0.43306269])

Next, we will obtain Alice's ratings to item 1 and item 4.
We already obtained `mod_rating_matrix`, and so we can easily get the rating scores as follows:

In [11]:
# In mod_rating_matrix, Alice is correspondent to 0th row, item 1 and item 4 are correspondent to the 0th column and the 3rd column, respectively.
mod_rating_matrix[0, [0, 3]]

array([1., 0.])

Now we are ready to predict Alice rating score for item 5.
Let's predict the score by using the above-mentioned equation.

In [12]:
mean_vec[0, 0] + np.dot(sim_vec, mod_rating_matrix[0, [0, 3]]) / sum(sim_vec) 

4.650185238495805

## Generalization for the above calculation
The above calculation is limited for predicting Alice's rating for item 5 using the two specific similar items.
For generalization, I prepared the function to predict an arbitrary user's rating for an arbitrary item.
The function is defined as the `predicting_rating_with_k_nn` method of the `ItemBasedCF` class in the file `cf.py` in the `lib` directory.

Let's run the following code.

In [13]:
!wget -P lib https://raw.githubusercontent.com/trycycle/recommender-system-2020/main/lib/cf.py

--2020-12-28 14:16:56--  https://raw.githubusercontent.com/trycycle/recommender-system-2020/main/lib/cf.py
raw.githubusercontent.com (raw.githubusercontent.com) をDNSに問いあわせています... 151.101.228.133
raw.githubusercontent.com (raw.githubusercontent.com)|151.101.228.133|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 4712 (4.6K) [text/plain]
`lib/cf.py' に保存中

cf.py               100%[===================>]   4.60K  --.-KB/s    in 0s      

2020-12-28 14:16:56 (42.0 MB/s) - `lib/cf.py' へ保存完了 [4712/4712]



In [14]:
# Import ItemBasedCF class 
from lib.cf import ItemBasedCF 

ibcf = ItemBasedCF() # Generate an instance of ItemBasedCF class
ibcf.predict_rating_with_k_nn(df, 0, 4, k=2)

4.622379629716474

---
## Assignment 2
In this assignment, we will apply the collaborative filtering to **MovieLens Latest Datasets (small)**, which was used for assignment 1.
As assignment 1, load the MovieLens data into variable `ml_df` using the function `get_movie_lens_dataframe` and complete the following assignments.


### Assignment 2-1
User 413 in `ml_df` has not rate the following movies yet (numbers mean movie IDs).
```
unrated_movies = [5, 76, 83, 242, 319, 351, 391, 473, 492, 597, 618, 634, 659, 733, 779, 1105, 1236, 1642, 1804, 2315]
```

By applying **item-based collaborative filtering** (`predicting_rating_with_k_nn` function) into the dataset, show a list of the top 20 recommended movies for user 413 and their predicted rating scores.
Note that a threshold for selecting nearest neighbor items (k) is 20.

### Assignment 2-2
User 413 graded movie 401 as 3 in `ml_df`.
Assume that the user had not rated the movie yet and predict the rating score of user 413 for movie 401 using **item-based collaborative filtering**.
Also, calculate the absolute value of the delta between the actual value (3) and the predicted value.
Note that a threshold for selecting nearest neighbor items (k) is 20.

### Assignment 2-3
In assignment 2-2, you calculated the delta value in the setting where k = 20.
Conduct the same calculation while changing the threshold k from 1 to 200 by 10.
Then, check how the absolute delta changes.

### Assignment 2-4
Conduct the same calculations in assignment 2-3 using **user-based collaborative filtering**.
Then, check how the absolute delta changes.